In [31]:
%matplotlib inline

datapath = "../../data/dl1/obj_neural_code/"
flatdatapath = "../../data/dl1/obj_neural_code/all_flat/"
repath   = "../../data/dl1/obj_neural_code/rep/finetune/"
N = 1440

In [3]:
from time import time
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import scipy.spatial.distance
import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16
vgg = Vgg16()

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [11]:
from keras.models import Model

idx = [0,5,10,17,24,31,33,35,37]
# lambda_1
# maxpooling2d_1
# maxpooling2d_2
# maxpooling2d_3
# maxpooling2d_4
# maxpooling2d_5
# dense_1
# dense_2
# dense_3

def get_models(idx,base):
    layers  = []
    for i in range(len(idx)):
        print(base.layers[idx[i]].name)
        layers.append( base.layers[idx[i]] )

    models = []
    for layer in layers:
        models.append( Model(input=base.input,
                             output=base.get_layer(layer.name).output) )
    print('Done.')
    return layers,models

In [13]:
batch_size = 60
nbatches = int(N/batch_size)
batches = vgg.get_batches(datapath+'train', batch_size=batch_size)
val_batches = vgg.get_batches(datapath+'valid', batch_size=batch_size)

Found 1120 images belonging to 40 classes.
Found 160 images belonging to 40 classes.


In [14]:
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=10)

Epoch 1/10
1120/1120 [==============================] - 33s - loss: 3.6408 - acc: 0.2321 - val_loss: 1.1234 - val_acc: 0.6500
Epoch 2/10
1120/1120 [==============================] - 32s - loss: 1.3523 - acc: 0.6045 - val_loss: 0.7391 - val_acc: 0.7500
Epoch 3/10
1120/1120 [==============================] - 33s - loss: 0.9182 - acc: 0.7179 - val_loss: 0.5594 - val_acc: 0.8000
Epoch 4/10
1120/1120 [==============================] - 33s - loss: 0.6315 - acc: 0.7955 - val_loss: 0.4489 - val_acc: 0.8562
Epoch 5/10
1120/1120 [==============================] - 33s - loss: 0.4769 - acc: 0.8509 - val_loss: 0.3915 - val_acc: 0.8813
Epoch 6/10
1120/1120 [==============================] - 33s - loss: 0.5102 - acc: 0.8357 - val_loss: 0.4674 - val_acc: 0.8312
Epoch 7/10
1120/1120 [==============================] - 34s - loss: 0.4159 - acc: 0.8661 - val_loss: 0.5214 - val_acc: 0.8437
Epoch 8/10
1120/1120 [==============================] - 33s - loss: 0.4328 - acc: 0.8580 - val_loss: 0.3149 - val_acc:

In [17]:
base = vgg.model
layers,models = get_models(idx,base)

lambda_1
maxpooling2d_1
maxpooling2d_2
maxpooling2d_3
maxpooling2d_4
maxpooling2d_5
dense_1
dense_2
dense_5
Done.


In [42]:
D = []
D_sq = []
for layer,model in zip(layers,models):
    print('Processing model : ' + layer.name)
    R = []
    batches = vgg.get_batches(flatdatapath, batch_size=batch_size, shuffle=False)
    ti = time()
    for n in range(nbatches):
        imgs,_ = batches.next()       
        R.append(model.predict(imgs))
    R = np.asarray(R, dtype=float) 
    R.shape = [N,3,224,224]
    print(R.shape)
    np.save(repath +  layer.name, R)
        
    # compute distances
    R.shape = R.shape[0],-1
    d = scipy.spatial.distance.pdist(R, 'euclidean')
    d_sq = scipy.spatial.distance.squareform(d, force='no', checks=True)
    D.append(d)
    D_sq.append(d_sq)
    te = time() - ti
    print(te) 
    
np.save(repath + 'D', D)
np.save(repath + 'D_sq', D_sq)
print('Done.')

Processing model : lambda_1
Found 1440 images belonging to 1 classes.
(1440, 3, 224, 224)
227.999969959
Processing model : maxpooling2d_1
Found 1440 images belonging to 1 classes.


ValueError: total size of new array must be unchanged

In [41]:
R.shape 

(24, 60, 64, 112, 112)